In [1]:
# Демонстрация функционала эстимейторов на видео

In [2]:
import os, sys

In [3]:
base_dir = os.path.split(os.getcwd())[0]
os.chdir(os.path.join(base_dir))

In [4]:
import pickle as pk

import numpy as np
import cv2
import sys
import argparse
import time
import copy
from tqdm import tqdm
import yaml

import matplotlib.pyplot as plt

from cranpose.utils import ARUCO_DICT, display_pose, f_left_x_04, f_right_x_04
from cranpose.estimators import PoseSingle
from cranpose.cammovement import CameraMovement, AbsdiffComparison

In [5]:
from scipy.spatial.transform import Rotation as R

In [6]:
k_matrix = "calib/calibration_matrix_hikvision.npy"
d_coeff = "calib/distortion_coefficients_hikvision.npy"
config_filename = "jupyters/marker_poses_warehouse.yaml"

edge_len = 0.7
n_markers = 100

type_ = "DICT_5X5_100"

if ARUCO_DICT.get(type_, None) is None:
    print(f"ArUCo tag type '{type_}' is not supported")
    sys.exit(0)

aruco_dict_type = ARUCO_DICT[type_]
calibration_matrix_path = k_matrix
distortion_coefficients_path = d_coeff

k = np.load(k_matrix)
d = np.load(d_coeff)


with open(config_filename, encoding='utf8') as f:
    marker_poses = yaml.load(f, Loader=yaml.FullLoader)

# marker_poses = dict(
#     zip([id for id in range(len(cfg['poses']))], 
#         cfg['poses']))

# Изменим калибровки так как разрешение потока отличается от разрешения калибровочного видео
k_video = copy.deepcopy(k)
k_video[0,0] = k_video[0,0] * 1280/2688
k_video[1,1] = k_video[1,1] * 720/1520

k_video[0,2] = k_video[0,2] * 1280/2688
k_video[1,2] = k_video[1,2] * 720/1520

In [9]:
cammovement = CameraMovement(
    compare_class = AbsdiffComparison(
        nrows=4, ncols=4, blur_size=5,
        threshold = 10,
        movement_perc_threshold = 0.1,
    )
)

In [10]:
cap = cv2.VideoCapture("D:\\NKB\\cranepos_data\\Камера 4_05_12_2023 14.16.17 clip.mp4")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/10)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter("D:\\NKB\\cranepos_data\\exp cammov4vis kf Камера 4_05_12_2023 14.16.17 clip.mp4", cv2.VideoWriter_fourcc('M','P','4','V'), 30, (width, height))

estimator_video_front = PoseSingle(
    aruco_dict_type,
    -1,
    n_markers,
    marker_poses,
    edge_len,
    k_video,
    d,
    apply_kf = True,
    transition_coef = 50,
    observation_coef = 0.02,
    x_bias = 1.25,
    camera_movement=cammovement,
    # left_edge_weight_func  = f_left_x_04,
)

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

x_poses_front = []

for _ in tqdm(range(length)):
    ret, frame = cap.read()
    if ret == True:
        frame, pose, size = estimator_video_front(
            frame, vis_detections=True, vis_movement=False)
        output = frame
        if pose.shape!=():
            output = display_pose(output, pose[:-1,[3]])
            x_poses_front.append(pose[0][3])
        else:
            x_poses_front.append(-1)
            
        out.write(output)
    else: 
        break

# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == True:
#         frame, pose, size = estimator_video_front(frame, True)
#         output = frame
#         if pose.shape!=():
#             output = display_pose(output, pose[:-1,[3]])
#             x_poses_front.append(pose[0][3])
#         else:
#             x_poses_front.append(-1)
            
#         out.write(output)
#     else: 
#         break

cap.release()
out.release()

  0%|          | 13/6378 [00:01<05:51, 18.13it/s]

True
False
False
False
False
False
False


  0%|          | 23/6378 [00:01<03:23, 31.30it/s]

False
False
False
False
False
False
False
False
False
False


  1%|          | 33/6378 [00:01<02:49, 37.32it/s]

...

100%|█████████▉| 6376/6378 [07:11<00:00,  8.67it/s]

False
False


100%|█████████▉| 6377/6378 [07:11<00:00,  8.45it/s]

False
False


100%|██████████| 6378/6378 [07:13<00:00, 14.71it/s]


In [10]:
cap.release()
out.release()

In [21]:
cap = cv2.VideoCapture("D:\\NKB\\cranepos_data\\Камера 5_05_12_2023 14.16.17 clip.mp4")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/10)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter("D:\\NKB\\cranepos_data\\res Камера 5_05_12_2023 14.16.17 clip.mov", cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

estimator_video_rear = PoseSingle(
    aruco_dict_type,
    1,
    n_markers,
    marker_poses,
    edge_len,
    k_video,
    d,
    apply_kf = True,
    transition_coef = 50,
    observation_coef = 0.02,
    x_bias = -1.5,
    # right_edge_weight_func  = lambda x: np.tanh(10*(1-x-0.35)).clip(min=0),
)

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

x_poses_rear = []

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame, pose, size = estimator_video_rear(frame, True)
        output = frame
        if pose.shape!=():
            output = display_pose(output, pose[:-1,[3]])
            x_poses_rear.append(pose[0][3])
        else:
            x_poses_rear.append(-1)
            
        out.write(output)
    else: 
        break

cap.release()
out.release()

In [ ]:
cap = cv2.VideoCapture('data/render_distance_long/front.mp4')

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/10)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter('data/results/demo_front_nokf.mov', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width,height))

estimator_video_front = PoseSingle(
    aruco_dict_type,
    1,
    n_markers,
    marker_poses,
    edge_len,
    k,
    d,
    apply_kf = False,
    transition_coef = 50,
    observation_coef = 0.02,
    x_bias = 1.25,
    left_edge_weight_func  = f_left_x_04,
)

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

x_poses_front = []

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame, pose, size = estimator_video_front(frame, True)
        output = frame
        if pose.shape!=():
            output = display_pose(output, pose[:-1,[3]])
            x_poses_front.append(pose[0][3])
        else:
            x_poses_front.append(-1)
            
        out.write(output)
    else: 
        break

cap.release()
out.release()

In [ ]:
begin_move = 80
x_poses_front = np.array(x_poses_front)
plt.figure()
plt.plot(np.arange(0,len(x_poses_front[begin_move:])), x_poses_front[begin_move:])
plt.plot(np.arange(0,len(x_poses_front_gt)), x_poses_front_gt)
plt.show()

In [ ]:
begin_move = 80
end_move = 680
x_poses_front = np.array(x_poses_front)
plt.figure()
resid = x_poses_front_gt[:end_move] - x_poses_front[begin_move:end_move+begin_move]
plt.plot(np.arange(0,len(resid)), resid)
plt.grid(True)
plt.ylim(ymin=-1, ymax=1)
plt.show()

In [ ]:
cap = cv2.VideoCapture('data/render_distance_long/rear.mp4')

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/10)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter('data/results/demo_rear_nokf.mov', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width,height))

estimator_video_rear = PoseSingle(
    aruco_dict_type,
    -1,
    n_markers,
    marker_poses,
    edge_len,
    k,
    d,
    apply_kf = False,
    transition_coef = 50,
    observation_coef = 0.02,
    x_bias = -1.5,
    right_edge_weight_func  = lambda x: np.tanh(10*(1-x-0.35)).clip(min=0),
)

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

x_poses_rear = []

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame, pose, size = estimator_video_rear(frame, True)
        output = frame
        if pose.shape!=():
            output = display_pose(output, pose[:-1,[3]])
            x_poses_rear.append(pose[0][3])
        else:
            x_poses_rear.append(-1)
            
        out.write(output)
    else: 
        break

cap.release()
out.release()

In [ ]:
begin_move = 90
x_poses_rear = np.array(x_poses_rear)
plt.plot(np.arange(0,len(x_poses_rear[begin_move:])), x_poses_rear[begin_move:])
plt.plot(np.arange(0,len(x_poses_rear_gt)), x_poses_rear_gt)

plt.show()

In [ ]:
begin_move = 90
end_move = 610
x_poses_rear = np.array(x_poses_rear)
plt.figure()
resid = x_poses_rear_gt[:end_move] - x_poses_rear[begin_move:end_move+begin_move]
plt.plot(np.arange(0,len(resid)), resid)
plt.grid(True)
plt.ylim(ymin=-1, ymax=1)
plt.show()

In [ ]:
# Среднее
begin_move_front = 80
begin_move_rear = 90
mean_x = np.mean([np.array(x_poses_rear[begin_move_rear:600+begin_move_rear]), np.array(x_poses_front[begin_move_front:600+begin_move_front])], axis=0)

In [ ]:
plt.figure()
resid = x_poses_front_gt[:600] - mean_x[:600]
plt.plot(np.arange(0,len(resid)), resid)
plt.grid(True)
plt.ylim(ymin=-1, ymax=1)
plt.show()